#Kaggle Titanic dataset

Importing files

In [1]:
from google.colab import files
uploaded = files.upload ()

Saving train.csv to train.csv
Saving test.csv to test.csv


Setting up the path to save files

In [45]:
import os

TITANIC_PATH = os.path.join("datasets", "titanic")

In [10]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

Loading test and train data

In [12]:
train_data = load_titanic_data("/content/train.csv")
test_data = load_titanic_data("/content/test.csv")

Lets look at the data populating the top 5 rows of the dataframe

In [13]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Uncovering the missing values

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


The attributes have the following meaning:

Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.

Pclass: passenger class.

Name, Sex, Age: self-explanatory

SibSp: how many siblings & spouses of the passenger aboard the Titanic.

Parch: how many children & parents of the passenger aboard the Titanic.

Ticket: ticket id

Fare: price paid (in pounds)

Cabin: passenger's cabin number

Embarked: where the passenger embarked the Titanic

We will have to process the data to fill missing values.

---



Let's view the distribution of numerical data

In [15]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Let's now take a quick look at categories housed in a few of the numerical and categorical attributes

In [16]:
train_data['Survived'].value_counts()   

0    549
1    342
Name: Survived, dtype: int64

In [17]:
train_data['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [18]:
train_data['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [19]:
train_data['Embarked'].value_counts() #The Embarked attribute tells us where the passenger embarked: C=Cherbourg, Q=Queenstown, S=Southampton

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X[self.attribute_names]

Building a pipeline for numerical attributes

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
                         ('select_numeric', DataFrameSelector(['Age','SibSp','Parch','Fare'])),
                         ('imputer', SimpleImputer(strategy='median'))                         
])


In [25]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ],
       [38.    ,  1.    ,  0.    , 71.2833],
       [26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [28.    ,  1.    ,  2.    , 23.45  ],
       [26.    ,  0.    ,  0.    , 30.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ]])

Now beginning work on a categorical pipeline. The imputer used earlier for the numerical pipeline replaces missing values with the median value, as there is no median value for categorical data a class will have to be generated to fill the empty values in a different manner. The below class has been inspired from stackoverflow.com/questions/25239958. It takes the most frequently occuring values in each column and fills it in the empty space.

In [26]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                    index=X.columns)
    return self
  def transform(self, X, y=None):
    return X.fillna(self.most_frequent_)

In [27]:
from sklearn.preprocessing import OneHotEncoder

In [28]:
cat_pipeline = Pipeline([
                         ('select_cat', DataFrameSelector(['Pclass', 'Sex', 'Embarked'])),
                         ('imputer', MostFrequentImputer()),
                         ('cat_encoder', OneHotEncoder(sparse=False))
])

In [29]:
cat_pipeline.fit_transform(train_data)

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

Joining the numerical and categorical pipelines

In [30]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
                                                     ('num_pipeline', num_pipeline),
                                                     ('cat_pipeline', cat_pipeline)
])

The pipeline can now take raw numberical and categorical data and output numerical features that can be fed into any machine learning model

In [63]:
X = preprocess_pipeline.fit_transform(train_data)


setting up the labels variables

In [64]:
y = train_data['Survived']

Splitting the data into train and validation sets.

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

Training an SVC classifier

In [89]:
from sklearn.svm import SVC
svm_clf = SVC(gamma='auto')
svm_clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Now that the model is trained we can use it to make predictions on the test set

In [103]:
y_pred = svm_clf.predict(X_val)
print("Accuracy on validation data: ",accuracy_score(y_val,y_pred))

Accuracy on validation data:  0.7333333333333333


Determing the accuracy of the model

Testing the data on another model for possible improved results

In [105]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators= 100, random_state=42)
forest_clf = forest_clf.fit(X_train, y_train)
forest_pred = forest_clf.predict(X_val)
print("Accuracy on validation data: ",accuracy_score(y_val,forest_pred))

Accuracy on validation data:  0.8333333333333334


Using gridsearchCV to find parameters to improve model accuracy

In [106]:
from sklearn.model_selection import GridSearchCV

In [107]:
param_grid = { 
    'n_estimators': [200, 350, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV = GridSearchCV(estimator=forest_clf, param_grid=param_grid, cv= 5)
CV.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

Finding out the best parameters

In [108]:
CV.best_params_


{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 500}

Training the model on the optimal parameters

In [112]:
forest_clf1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 500, max_depth=6, criterion='entropy')

In [113]:
forest_clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

Accuracy of model predictions based on the optimal parameters

In [114]:
from sklearn.metrics import accuracy_score

pred=forest_clf1.predict(X_val)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_val,pred))

Accuracy for Random Forest on CV data:  0.8444444444444444
